In [1]:
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import sklearn.metrics

epochs = 50

/usr/lib/python3/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [3]:
trainSentences = readfile("data/train.txt")
devSentences = readfile("data/valid.txt")
testSentences = readfile("data/test.txt")
print(trainSentences[0])

[['EU', 'B-ORG\n'], ['rejects', 'O\n'], ['German', 'B-MISC\n'], ['call', 'O\n'], ['to', 'O\n'], ['boycott', 'O\n'], ['British', 'B-MISC\n'], ['lamb', 'O\n'], ['.', 'O\n']]


In [4]:
trainSentences = addCharInformatioin(trainSentences)
devSentences = addCharInformatioin(devSentences)
testSentences = addCharInformatioin(testSentences)

In [20]:
print(trainSentences[0])

[['EU', ['E', 'U'], 'B-ORG\n'], ['rejects', ['r', 'e', 'j', 'e', 'c', 't', 's'], 'O\n'], ['German', ['G', 'e', 'r', 'm', 'a', 'n'], 'B-MISC\n'], ['call', ['c', 'a', 'l', 'l'], 'O\n'], ['to', ['t', 'o'], 'O\n'], ['boycott', ['b', 'o', 'y', 'c', 'o', 't', 't'], 'O\n'], ['British', ['B', 'r', 'i', 't', 'i', 's', 'h'], 'B-MISC\n'], ['lamb', ['l', 'a', 'm', 'b'], 'O\n'], ['.', ['.'], 'O\n']]


In [5]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token.lower()] = True

In [6]:
print(labelSet)

{'B-MISC\n', 'B-ORG\n', 'I-MISC\n', 'B-PER\n', 'I-PER\n', 'I-LOC\n', 'O\n', 'B-LOC\n', 'I-ORG\n'}


In [23]:
print(words)

{'realistic': True, 'plead': True, 'jittery': True, 'widened': True, 'emese': True, 'soviet-style': True, 'disregard': True, '0931': True, 'lankans': True, '2:43.06': True, 'deny': True, 'hamid': True, 'syria': True, 'roadside': True, 'fertile': True, '106-26/32': True, 'amusement': True, 'constructors': True, 'ong': True, '13.60': True, 'bordeaux': True, 'theft': True, 'sikhin': True, 'credible': True, 'practising': True, 'volendam': True, 'musa': True, 'dated': True, 'people': True, 'cyberspace': True, 'curran': True, 'zion': True, 'helens': True, 'poles': True, '7340': True, 'realize': True, 'france-registered': True, 'arable': True, 'refuse': True, 'protect': True, '8:15.69': True, 'waugh': True, '74,600': True, "o'clock": True, 'person': True, 'turbulent': True, 'loine': True, 'bogeys': True, 'schedules': True, '3-4': True, '343-8': True, 'withdrawals': True, 'scrambled': True, '16.63': True, 'campaigning': True, 'literature': True, 'concessions': True, 'samples': True, 'effort': 

In [7]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [8]:
print(label2Idx)

{'B-PER\n': 3, 'I-PER\n': 4, 'I-LOC\n': 5, 'B-ORG\n': 1, 'B-LOC\n': 7, 'O\n': 6, 'B-MISC\n': 0, 'I-MISC\n': 2, 'I-ORG\n': 8}


In [9]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [10]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'contains_digit': 6, 'allUpper': 2, 'allLower': 1, 'numeric': 0, 'mainly_numeric': 5, 'initialUpper': 3, 'PADDING_TOKEN': 7, 'other': 4}


In [11]:
# :: Read in word embeddings ::
word2Idx = {}
wordEmbeddings = []

fEmbeddings = open("embeddings/glove.6B.100d.txt", encoding="utf-8")
# 6 billion words. Pretrained. Each with 100 dimensional word embedding. eg
'''
    the 0.04656 0.21318 -0.0074364 -0.45854 -0.035639 0.23643 -0.28836 0.21521 -0.13486 -1.6413 -0.26091 0.032434 0.056621 -0.043296 -0.021672 0.22476 -0.075129 -0.067018 -0.14247 0.038825 -0.18951 0.29977 0.39305 0.17887 -0.17343 -0.21178 0.23617 -0.063681 -0.42318 -0.11661 0.093754 0.17296 -0.33073 0.49112 -0.68995 -0.092462 0.24742 -0.17991 0.097908 0.083118 0.15299 -0.27276 -0.038934 0.54453 0.53737 0.29105 -0.0073514 0.04788 -0.4076 -0.026759 0.17919 0.010977 -0.10963 -0.26395 0.07399 0.26236 -0.1508 0.34623 0.25758 0.11971 -0.037135 -0.071593 0.43898 -0.040764 0.016425 -0.4464 0.17197 0.046246 0.058639 0.041499 0.53948 0.52495 0.11361 -0.048315 -0.36385 0.18704 0.092761 -0.11129 -0.42085 0.13992 -0.39338 -0.067945 0.12188 0.16707 0.075169 -0.015529 -0.19499 0.19638 0.053194 0.2517 -0.34845 -0.10638 -0.34692 -0.19024 -0.2004 0.12154 -0.29208 0.023353 -0.11618 -0.35768 0.062304 0.35884 0.02906 0.0073005 0.0049482 -0.15048 -0.12313 0.19337 0.12173 0.44503 0.25147 0.10781 -0.17716 0.038691 0.08153 0.14667 0.063666 0.061332 -0.075569 -0.37724 0.01585 -0.30342 0.28374 -0.042013 -0.040715 -0.15269 0.07498 0.15577 0.10433 0.31393 0.19309 0.19429 0.15185 -0.10192 -0.018785 0.20791 0.13366 0.19038 -0.25558 0.304 -0.01896 0.20147 -0.4211 -0.0075156 -0.27977 -0.19314 0.046204 0.19971 -0.30207 0.25735 0.68107 -0.19409 0.23984 0.22493 0.65224 -0.13561 -0.17383 -0.048209 -0.1186 0.0021588 -0.019525 0.11948 0.19346 -0.4082 -0.082966 0.16626 -0.10601 0.35861 0.16922 0.07259 -0.24803 -0.10024 -0.52491 -0.17745 -0.36647 0.2618 -0.012077 0.08319 -0.21528 0.41045 0.29136 0.30869 0.078864 0.32207 -0.041023 -0.1097 -0.092041 -0.12339 -0.16416 0.35382 -0.082774 0.33171 -0.24738 -0.048928 0.15746 0.18988 -0.026642 0.063315 -0.010673 0.34089 1.4106 0.13417 0.28191 -0.2594 0.055267 -0.052425 -0.25789 0.019127 -0.022084 0.32113 0.068818 0.51207 0.16478 -0.20194 0.29232 0.098575 0.013145 -0.10652 0.1351 -0.045332 0.20697 -0.48425 -0.44706 0.0033305 0.0029264 -0.10975 -0.23325 0.22442 -0.10503 0.12339 0.10978 0.048994 -0.25157 0.40319 0.35318 0.18651 -0.023622 -0.12734 0.11475 0.27359 -0.21866 0.015794 0.81754 -0.023792 -0.85469 -0.16203 0.18076 0.028014 -0.1434 0.0013139 -0.091735 -0.089704 0.11105 -0.16703 0.068377 -0.087388 -0.039789 0.014184 0.21187 0.28579 -0.28797 -0.058996 -0.032436 -0.0047009 -0.17052 -0.034741 -0.11489 0.075093 0.099526 0.048183 -0.073775 -0.41817 0.0041268 0.44414 -0.16062 0.14294 -2.2628 -0.027347 0.81311 0.77417 -0.25639 -0.11576 -0.11982 -0.21363 0.028429 0.27261 0.031026 0.096782 0.0067769 0.14082 -0.013064 -0.29686 -0.079913 0.195 0.031549 0.28506 -0.087461 0.0090611 -0.20989 0.053913
    of -0.076947 -0.021211 0.21271 -0.72232 -0.13988 -0.12234 -0.17521 0.12137 -0.070866 -1.5721 -0.22464 0.04269 -0.4018 0.21006 0.014288 0.41628 0.017165 0.071732 0.0069246 0.18107 -0.15412 0.14933 -0.030493 0.29918 0.029479 -0.036147 -0.061125 0.083918 -0.12398 -0.10077 -0.0054142 0.3371 -0.25612 0.44388 -0.68922 0.1802 0.34898 -0.052284 -0.26226 -0.47109 0.21647 -0.4002 -0.049986 0.011376 0.54994 -0.22791 0.095873 0.47693 -0.056727 -0.17895 0.11756 0.14662 0.048948 0.13587 -0.093821 0.45968 -0.32062 0.29911 0.20656 -0.18503 -0.2769 -0.022545 0.70698 -0.23815 0.16437 -0.55044 -0.0010615 0.12266 0.11898 0.23985 0.29815 0.013207 0.16316 -0.61334 -0.37051 0.19444 -0.13621 -0.30426 -0.37715 0.065299 -0.15995 -0.56516 0.074696 0.40184 0.19328 0.041802 0.20572 0.28971 0.34783 0.33873 -0.10052 -0.16397 -0.15236 -0.086815 0.36522 0.14969 -0.40859 0.23106 0.17162 -0.60545 0.086019 0.37043 0.17937 -0.40282 -0.62471 -0.055919 0.15092 0.12554 -0.45344 0.34417 0.40042 -0.049512 -0.29969 -0.31761 0.30023 0.090029 0.3106 -0.033077 -0.21995 -0.40396 -0.34443 -0.21248 -0.37636 0.21835 -0.1785 -0.17261 0.16391 0.22753 0.2686 0.57541 -0.14912 0.20413 0.22187 -0.27014 0.068253 0.29115 -0.067943 0.10623 -0.16281 0.19939 -0.48613 0.035688 -0.12373 0.13707 0.33359 -0.12713 -0.31711 -0.13962 -0.04288 -0.0014614 0.76883 -0.41705 -0.092911 0.16315 0.29202 0.12119 -0.076683 0.14131 -0.093406 -0.042796 0.13738 0.014278 0.11918 -0.34215 -0.19076 -0.12499 0.24648 0.42259 0.091966 0.45351 0.14437 0.1878 -0.85876 0.059621 -0.32242 0.28627 0.12427 0.0090984 -0.1891 0.16638 0.099881 -0.048553 -0.026257 0.099904 0.12406 -0.015416 -0.29707 -0.4044 -0.17258 0.36468 -0.014118 -0.11889 -0.11686 -0.14124 0.28012 0.067644 0.1485 -0.35702 0.29626 0.36004 1.019 -0.067307 -0.11588 -0.2178 0.070191 0.23154 -0.13849 0.26441 0.28742 0.1941 -0.0060504 0.44105 0.12416 -0.27745 -0.25729 0.10992 0.18362 -0.34522 -0.21861 -0.18825 -0.037454 -0.20862 -0.25216 0.060842 0.068595 0.10275 0.10745 -0.061288 0.19725 -0.27739 -0.022559 0.052794 -0.24083 0.09199 0.30959 0.054999 0.063676 -0.087357 -0.34495 0.22793 -0.42405 0.24536 0.55708 0.19126 -0.797 -0.2048 0.32545 0.09235 0.084791 -0.16433 -0.066568 -0.099249 0.31526 -0.44465 0.087281 0.3288 -0.017809 -0.23855 -0.12848 0.041509 0.46728 0.48214 0.10548 0.065805 0.067221 0.13321 -0.27856 0.015532 0.30026 0.38748 -0.14401 -0.16131 0.17678 0.16448 -0.3244 0.007937 -2.2836 0.096945 0.66131 0.16857 -0.028877 -0.10791 -0.027445 -0.25695 0.046686 0.23087 -0.076458 0.27127 0.25185 0.054947 -0.36673 -0.38603 0.3029 0.015747 0.34036 0.47841 0.068617 0.18351 -0.29183 -0.046533

'''

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)

        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0].lower() in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[split[0]] = len(word2Idx)
        
wordEmbeddings = np.array(wordEmbeddings)

In [15]:
print(word2Idx)


NameError: name 'word2Idx' is not defined

In [47]:
print(wordEmbeddings[2]) # gives the word embedding(100 dimensional corresponding to every word. The index in wordEmbeddings 
                         # corresponds to the index of the word in word to id.
print(len(wordEmbeddings[0]))

[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.27062 ]

In [12]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|":
    char2Idx[c] = len(char2Idx)

In [13]:
print(char2Idx)

{'o': 27, 'e': 17, 'I': 47, 'A': 39, 'v': 34, '/': 82, '1': 4, 'C': 41, 'P': 54, 'H': 46, 'w': 35, ',': 66, 'B': 40, 'M': 51, 'W': 61, 'D': 42, '.': 65, 'y': 37, 'h': 20, 'T': 58, 'z': 38, 'k': 23, '3': 6, 'O': 53, 'J': 48, 'UNKNOWN': 1, 'm': 25, '#': 79, 'q': 29, "'": 80, '+': 90, '{': 73, '@': 91, '8': 11, '_': 68, '^': 92, '*': 89, 's': 31, 'c': 15, 'r': 30, 'j': 22, 'L': 50, 'N': 52, 'F': 44, '-': 67, '(': 69, '=': 88, '}': 74, '$': 85, '[': 71, '~': 93, 'R': 56, ']': 72, '`': 86, ';': 78, 't': 32, ' ': 2, '0': 3, 'X': 62, 'S': 57, '4': 7, 'G': 45, 'l': 24, 'p': 28, ':': 77, 'd': 16, '"': 81, 'U': 59, '6': 9, ')': 70, 'n': 26, 'K': 49, 'f': 18, '?': 76, 'g': 19, 'Y': 63, 'a': 13, '9': 12, '\\': 83, 'x': 36, '7': 10, '!': 75, '%': 84, 'u': 33, 'i': 21, 'E': 43, '|': 94, 'Q': 55, '5': 8, '&': 87, 'V': 60, 'PADDING': 0, '2': 5, 'Z': 64, 'b': 14}


In [14]:
print(trainSentences[0])

[['EU', ['E', 'U'], 'B-ORG\n'], ['rejects', ['r', 'e', 'j', 'e', 'c', 't', 's'], 'O\n'], ['German', ['G', 'e', 'r', 'm', 'a', 'n'], 'B-MISC\n'], ['call', ['c', 'a', 'l', 'l'], 'O\n'], ['to', ['t', 'o'], 'O\n'], ['boycott', ['b', 'o', 'y', 'c', 'o', 't', 't'], 'O\n'], ['British', ['B', 'r', 'i', 't', 'i', 's', 'h'], 'B-MISC\n'], ['lamb', ['l', 'a', 'm', 'b'], 'O\n'], ['.', ['.'], 'O\n']]


In [15]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

In [16]:
# train-set[0]: corresponds to the ids of the words in the sentence
# train_set[1]: corresponds to the ids of the cases of the words
# train_set[2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0]))
print(len(train_set[0][2]))

[[641, 6732, 512, 578, 6, 4940, 295, 8353, 4], [2, 1, 3, 1, 1, 1, 3, 1, 4], array([[43, 59,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [30, 17, 22, 17, 15, 32, 31,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [45, 17, 30, 25, 13, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [15, 13, 24, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  

In [17]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [89]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
# words2 = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=True)(words_input)

casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words,  casing,char])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 2850        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_13 (Dropout)            (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_25 (TimeDistri (None, None, 52, 30) 2730        dropout_13[0][0]                 
__________________________________________________________________________________________________
time_distr

In [90]:
test_results = 0

In [96]:

for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch       
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
        print(' ')


Epoch 0/50
63/64 [============================>.] - ETA: 0s 
Epoch 1/50
63/64 [============================>.] - ETA: 0s 
Epoch 2/50
32/64 [==============>...............] - ETA: 8s 


63/64 [============================>.] - ETA: 0s 
Epoch 3/50
63/64 [============================>.] - ETA: 0s 
Epoch 4/50
63/64 [============================>.] - ETA: 0s 
Epoch 5/50
 2/64 [..............................] - ETA: 9s  


63/64 [============================>.] - ETA: 0s 
Epoch 6/50
63/64 [============================>.] - ETA: 0s 
Epoch 7/50
36/64 [===============>..............] - ETA: 7s 


63/64 [============================>.] - ETA: 0s 
Epoch 8/50
63/64 [============================>.] - ETA: 0s 
Epoch 9/50
63/64 [============================>.] - ETA: 0s 
Epoch 10/50
 6/64 [=>............................] - ETA: 11s 


63/64 [============================>.] - ETA: 0s 
Epoch 11/50
63/64 [============================>.] - ETA: 0s 
Epoch 12/50
40/64 [=================>............] - ETA: 6s 


...

63/64 [============================>.] - ETA: 0s 
Epoch 38/50
63/64 [============================>.] - ETA: 0s 
Epoch 39/50
63/64 [============================>.] - ETA: 0s 
Epoch 40/50
26/64 [===========>..................] - ETA: 9s 


63/64 [============================>.] - ETA: 0s 
Epoch 41/50
63/64 [============================>.] - ETA: 0s 
Epoch 42/50
60/64 [===========================>..] - ETA: 0s 


63/64 [============================>.] - ETA: 0s 
Epoch 43/50
63/64 [============================>.] - ETA: 0s 
Epoch 44/50
63/64 [============================>.] - ETA: 0s 
Epoch 45/50
29/64 [============>.................] - ETA: 9s 


63/64 [============================>.] - ETA: 0s 
Epoch 46/50
63/64 [============================>.] - ETA: 0s 
Epoch 47/50
63/64 [============================>.] - ETA: 0s 
Epoch 48/50


63/64 [============================>.] - ETA: 0s 
Epoch 49/50
63/64 [============================>.] - ETA: 0s 


In [97]:
predLabels, correctLabels = tag_dataset(dev_batch)  
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()


print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))

3249/3250 [============================>.] - ETA: 0s0.913571954467381
0.984015419960282
0.9838808149926056


In [98]:
# predLabels, correctLabels = tag_dataset(dev_batch)  
predLabels, correctLabels = tag_dataset(test_batch)
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
f1_macro = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
print(f1_macro)
# print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

3452/3453 [============================>.] - ETA: 0s0.8709413378342556
0.9748250242274147
0.9749878656624975
[0.81450488 0.85856079 0.66666667 0.94428883 0.97544162 0.82952548
 0.99262007 0.90856648 0.84829721]


In [94]:
test_results+=f1_macro

In [99]:
print(test_results)


0.8711842982997874


In [51]:
model.save('conll_with_char.h5')

In [52]:
print(len(word2Idx))

22949


In [53]:
print(len(trainSentences)+len(testSentences)+len(devSentences))

20744
